In [1]:
## change to our functinon
from stage import getStage, fullPrint
import numpy as np
import pandas as pd
from datetime import date, timedelta, datetime
import yahoo_fin.stock_info as yf
import random
SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
sectorDfList = []
for sector in SectorDict.values():
    sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = set(nasdaqList + nyseList)
def evalOneMax(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    print(individual[0])
    winningTrade = 0
    totalTrade = 0
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    goodSectorDf = sectorDfList[0].drop(['close','ticker','MA30','MA30Slope','RS'],axis=1)
    goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
    goodSectorDf.index = sectorDfList[0].index
    for date in sectorDfList[0].index:
        listOfSector = []
        for df in sectorDfList:
            try:
                if df.at[date,'MA30Slope'] > individual[0][9] and df.loc[date,'RS'] > individual[0][10]:
                    if df.at[date,'close'] > df.at[date,'MA30']:
                        listOfSector.append(df.at[date,'ticker'])
            except: 
                pass
        goodSectorDf.at[date,'Sectors'] = set(listOfSector)
    for symbol in nasdaqList:
        df = getStage(symbol,individual[0],goodSectorDf)
        inStage = False
        buyTwice = False
        if df.empty:
            continue
        buyPrice = 0
        secondBuyPrice = 0
        drift = -1
        for index, element in df.iterrows():
            if drift == -1:
                drift = index.weekday()
            index = index - timedelta(days= drift)
            index = index.strftime('%Y-%m-%d')
            if element.Stage == "Stage 2" or element.Stage == "Buy":
                if element.Stage == "Buy" and not inStage:
                    buyPrice = element.close
                transaction.loc[index]['holding'].append((symbol,element.close,0))
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                if element.Stage == "Buy" and inStage:
                    secondBuyPrice = element.close
                    transaction.loc[index]['holding'].append((symbol,element.close,0))
                    buyTwice = True
                inStage = True
                continue
            if element.Stage == "Sell":
                transaction.loc[index]['holding'].append((symbol,element.close,-1))
                returnRate = element.close / buyPrice
                if returnRate > 1:
                    winningTrade += 1
                    totalTrade += 1
                else:
                    totalTrade += 1
                if buyTwice:
                    transaction.loc[index]['holding'].append((symbol,element.close,-1))
                    returnRate = element.close / secondBuyPrice
                    if returnRate > 1:
                        winningTrade += 1
                        totalTrade += 1
                    else:
                        totalTrade += 1
                buyPrice = 0
                secondBuyPrice = 0
                inStage = False
                buyTwice = False
    total = 100
    prevIndex = ''
    first = True
    transactionCopy = transaction
    transactionCopy['total'] = 0
    for index, element in transactionCopy.iterrows():
        index = index.strftime('%Y-%m-%d')
        if len(element.holding) == 0:
            transactionCopy['total'][index] = total
            continue
        if first:
            first = False
            transactionCopy['total'][index] = total
        else:
            prevData = transactionCopy.loc[prevIndex].holding
            if len(prevData):
                total = 0
                removeList = []
                close = 0
                for i in range(len(prevData)):
                    for j in range(len(element.holding)):
                        if element.holding[j][0] == prevData[i][0]:
                            close = element.holding[j][1]
                            if element.holding[j][2] == -1:
                                removeList.append(element.holding[j])
                            break
                    total += close*prevData[i][2]
                transactionCopy['total'][index] = total
                for delete in removeList:
                    if delete in element.holding:
                        element.holding.remove(delete)
            else:
                transactionCopy['total'][index] = transactionCopy.loc[prevIndex].total
        if len(element.holding):
            allocation = total/len(element.holding)
        for i in range(len(element.holding)):
            element.holding[i] = (element.holding[i][0],element.holding[i][1],allocation/element.holding[i][1])
        prevIndex = index
    stockHolding  = 0
    for i in transactionCopy.iterrows():
        stockHolding += len(i[1]['holding'])
    print(stockHolding/(transactionCopy.shape[0]))
    print(transactionCopy.iloc[-1]['total'])
    if totalTrade==0:
        print(winningTrade, totalTrade, 0)
    else:
        print(winningTrade, totalTrade, winningTrade/totalTrade)
    if stockHolding/(transactionCopy.shape[0])<8:
        return 0,
    return transactionCopy.iloc[-1]['total']
# evalOneMax([[0.2088062403011085, 1.3923948213253003, 0.35063207272053876, 0.9325315828692986, 1.0884748465365186, 1.2178645745962666, 0.976399694816296, 0.9766178952891444, 1.383289733820422, 0.9953218464817897, -0.014234317027519955, 0.9663375876834599, 0.9940289768224931, 0.9626303277850059, 0.9844597312889762]])

In [2]:
import ast
file = open('resultAlan.txt')
i = 0
coef = ""
profit = 0
listOfCoef = []
for line in file:
    if "--" in line:
        continue
    if i%4 == 0:
        try:
            coef = ast.literal_eval(line.strip('\n'))
        except:
            pass
        i = i + 1
    elif i%4 == 2:
        profit = float(line)
        if profit > 30000 and profit < 32000:
            listOfCoef.append(coef)
        i = i + 1
    else:
        i = i + 1
print(listOfCoef)

[[0.08377026152663691, 0.8369486592147836, 0.3216617105756667, 0.9846501272634265, 1.0760786970329437, 1.2156368313224644, 0.9160035446527982, 0.9875510696777539, 1.337495141974483, 0.9952010119205937, 0.039984229134584215, 0.9840462801650771, 0.9873513441459263, 0.948039045982513, 0.9847277794993363], [0.24741802376193045, 1.1969293168070645, 0.3919744546518519, 0.9212277434136174, 1.2447995291169438, 1.2400322562034027, 0.970549470415594, 0.8614402727446571, 1.353668816788288, 1.0020146700186787, 0.0329932613652328, 0.9822545771978067, 0.9898745397449472, 0.9496857678547218, 0.9844684744298214], [0.1455292612654837, 1.0657283976488348, 0.2599120891097705, 0.9368252123743406, 1.0343094168251652, 1.2059367621212709, 0.991722454551891, 0.8168152723361571, 1.2888835028347978, 0.9937256958102058, 0.014657575281017729, 0.9809445586120963, 0.9925082406439346, 0.9537582814003209, 0.9908883730284089], [0.0766279044925956, 0.6596961889877901, -0.07384632725402261, 0.983404769117893, 1.24362566

In [3]:
for i in range(len(listOfCoef)):
    print(listOfCoef[i][0])
    evalOneMax([listOfCoef[i]])

0.08377026152663691
[0.08377026152663691, 0.8369486592147836, 0.3216617105756667, 0.9846501272634265, 1.0760786970329437, 1.2156368313224644, 0.9160035446527982, 0.9875510696777539, 1.337495141974483, 0.9952010119205937, 0.039984229134584215, 0.9840462801650771, 0.9873513441459263, 0.948039045982513, 0.9847277794993363]


In [2]:
from stage import getFullDf
getFullDf("ABUS",[0.3318662099030736, 1.7347893537959698, 0.1916285863344682, 0.9350007642794158, 1.3283762780182027, 1.0069983167660093, 0.9708546875087183, 0.8212695161601586, 1.4036713065821127]).to_csv("ABUS.csv")

6484


In [3]:
# from stage import getStage
# getStage("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544])
from stage import getFullDf
getFullDf("AAPL",[0.3866447912682057, 1.554754873864911, 0.11941121685460436, 1.0326272714103917, 1.0873323087402937, 1.0382277515512988, 0.9779939752154027, 0.9698535499223544,1.5]).to_csv("result.csv")

6484
